Tensorboard, Saver
===========

Saver
---

In [5]:
# 모델을 저장하고 재사용하는 방법

import tensorflow as tf
import numpy as np

data = np.loadtxt('data/data.csv', delimiter = ',',
                 unpack = True, dtype = 'float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

# 신경망 모델 구성
# 학습에 직접적으로 사용하지 않고 학습 횟수에 따라 단순히 증가시킬 변수를 만듭니다.
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model)
)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

# global_step으로 넘겨준 변수를 학습용 변수들을 최적화할 때마다 학습 횟수를 하나씩 증가시킵니다.
train_op = optimizer.minimize(cost, global_step = global_step)

# 신경망 모델 학습
sess = tf.Session()
# 모델을 저장하고 불러오는 API를 초기화
# global_variable 함수를 통해 앞서 정의하였던 변수들을 저장하거나 불러올 변수들로 설정함
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('data/model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restores(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
for step in range(2):
    sess.run(train_op, feed_dict = {X : x_data, Y : y_data})
    
    print('Step :', sess.run(global_step),
         'Cost :', sess.run(cost, feed_dict = {X : x_data, Y : y_data}))
    
# 최적화가 끝난 뒤 변수를 저장
saver.save(sess, 'data/model/dnn.ckpt', global_step = global_step)

# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Step : 1 Cost : 0.69759804
Step : 2 Cost : 0.6571211
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


TensorBoard
========